<a href="https://colab.research.google.com/github/Uttumon/Big_Data_Analytics/blob/main/BDA2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import random
import sqlite3
import pandas as pd
from collections import defaultdict

def generate_sample_data(n_records=10000):
    data = []
    for i in range(n_records):
        record = {
            'id': i,
            'name': f'User_{i}',
            'age': random.randint(18, 80),
            'salary': random.uniform(30000, 150000),
            'city': random.choice(['New York', 'London', 'Tokyo', 'Sydney'])
        }
        data.append(record)
    return data

class SQLiteDB:
    def __init__(self):
        self.conn = sqlite3.connect(':memory:')  # In-memory database
        self.cursor = self.conn.cursor()

    def setup(self):
        # Separate the DROP and CREATE statements
        self.cursor.execute("DROP TABLE IF EXISTS employees")
        self.cursor.execute("""
            CREATE TABLE employees (
                id INTEGER PRIMARY KEY,
                name TEXT,
                age INTEGER,
                salary REAL,
                city TEXT
            )
        """)
        self.conn.commit()

    def insert_data(self, data):
        start_time = time.time()
        for record in data:
            self.cursor.execute("""
                INSERT INTO employees (id, name, age, salary, city)
                VALUES (?, ?, ?, ?, ?)
            """, (record['id'], record['name'], record['age'],
                  record['salary'], record['city']))
        self.conn.commit()
        return time.time() - start_time

    def query_data(self):
        # Sample query: Get employees with salary > 100000
        start_time = time.time()
        self.cursor.execute("""
            SELECT * FROM employees
            WHERE salary > 100000
        """)
        results = self.cursor.fetchall()
        return time.time() - start_time, len(results)

    def close(self):
        self.conn.close()

class MockMongoDB:
    def __init__(self):
        self.data = defaultdict(dict)

    def setup(self):
        self.data.clear()

    def insert_data(self, data):
        start_time = time.time()
        for record in data:
            self.data[record['id']] = record
        return time.time() - start_time

    def query_data(self):
        # Same query as SQL: Get employees with salary > 100000
        start_time = time.time()
        results = [doc for doc in self.data.values() if doc['salary'] > 100000]
        return time.time() - start_time, len(results)

    def close(self):
        self.data.clear()

def evaluate_performance(n_records=10000):
    # Generate sample data
    data = generate_sample_data(n_records)

    # Test SQLite (SQL)
    sql_db = SQLiteDB()
    sql_db.setup()

    sql_insert_time = sql_db.insert_data(data)
    sql_query_time, sql_result_count = sql_db.query_data()

    sql_db.close()

    # Test Mock MongoDB (NoSQL)
    mongo_db = MockMongoDB()
    mongo_db.setup()

    mongo_insert_time = mongo_db.insert_data(data)
    mongo_query_time, mongo_result_count = mongo_db.query_data()

    mongo_db.close()

    results = {
        'Database': ['SQLite (BigSQL)', 'Mock MongoDB (NoSQL)'],
        'Insert Time (s)': [sql_insert_time, mongo_insert_time],
        'Query Time (s)': [sql_query_time, mongo_query_time],
        'Results Found': [sql_result_count, mongo_result_count]
    }

    df = pd.DataFrame(results)
    print(f"\nPerformance Comparison with {n_records} records:")
    print(df)

# Run the evaluation
if __name__ == "__main__":
    try:
        evaluate_performance(10000)
    except Exception as e:
        print(f"An error occurred: {str(e)}")


Performance Comparison with 10000 records:
               Database  Insert Time (s)  Query Time (s)  Results Found
0       SQLite (BigSQL)         0.024256        0.006657           4178
1  Mock MongoDB (NoSQL)         0.001009        0.000902           4178
